In [ ]:
from ase.io import read as ase_read
from ase.io import write as ase_write
import numpy as np
import copy
import json

In [ ]:
input_xyz = "jp_dio-orig_min4.xyz"
min_orig_atoms = ase_read(input_xyz)

In [ ]:
with open("noOidx2orig.json", "r") as f:
    index_map = json.load(f)

# I want to reverse this, i.e. go from orig to noO
index_map = {int(v): int(k) for k,v in index_map.items()}

In [ ]:
grain_ptm_data = np.load("grains_ptm_111025_min4_fixed.npz") # notice! using the fixed version now
noO_grains = grain_ptm_data["grains"]
noO_ptm_types = grain_ptm_data["ptm_types"]

In [ ]:
xyz_grain_idxs = []
xyz_ptm_types = []

for i,atm in enumerate(min_orig_atoms):
    if atm.symbol == "O":
        xyz_grain_idxs.append(-1)
        xyz_ptm_types.append(-1)
        continue

    xyz_grain_idxs.append(noO_grains[index_map[i]])
    xyz_ptm_types.append(noO_ptm_types[index_map[i]])

In [ ]:
with open("proposed_selected_envs_v1.json", "r") as f:
    selected_envs = json.load(f)

In [ ]:
selected_envs

In [ ]:
selected_idxs = [env["index"] for env_list in selected_envs.values() for env in env_list]

In [ ]:
selected_idxs

In [ ]:
def generate_selected_temp_xyz(selected_idxs):
    min_orig_out = copy.deepcopy(min_orig_atoms)

    new_symbols = min_orig_out.get_chemical_symbols().copy()
    for i in selected_idxs:
        new_symbols[i] = "Np"
    min_orig_out.set_chemical_symbols(new_symbols)

    min_orig_out.set_array("grain_index", np.array(xyz_grain_idxs))
    min_orig_out.set_array("ptm_type", np.array(xyz_ptm_types))

    ase_write("temp_selected.xyz", min_orig_out, format="extxyz")


In [ ]:
generate_selected_temp_xyz(selected_idxs)

In [ ]:
with open("200_select_Gb_oint_indices.txt", "w") as f:
    for i in selected_idxs:
        f.write(f"{i}\n")